In [1]:
# Instruções basiconas de sempre.

# Lidar com dataframes.
import numpy as np
import pandas as pd

# Lidar com gráficos.
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from plotting import (multiple_histograms_plot,
                      bar_plot_with_categorical,
                      plot_confusion_matrix,
                      plot_confusion_matrix_2,
                      plot_roc)

# Lidar com preparação de dados.
from data_prep import data_prep as dp # Eu que fiz esse modulinho ("uuuuuuuuuma bosts!").
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (StandardScaler
                                   , MinMaxScaler)
from sklearn.model_selection import (train_test_split
                                     , cross_val_score
                                     , StratifiedKFold)

# Lidar com validação de modelos.
from sklearn.metrics import (confusion_matrix
                             , accuracy_score
                             , classification_report)

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)

!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [2]:
#Bibliotecas necessárias para aquisitar as informações em json em requisitando uma API Rest.

import json
import requests

# Aquisitando os dados através de requisições a API do Brasil.IO e transformando JSON em um DataFrame
Seguindo as instruções em uma questão do [Stackoverflow](https://stackoverflow.com/questions/21104592/json-to-pandas-dataframe), temos:

In [4]:
# Requisições por número da página

df_gastos_parlamentares = pd.DataFrame(list())

for numero in range (1,101):
    r = requests.get("https://brasil.io/api/dataset/gastos-deputados/cota_parlamentar/data?page="+str(numero))
    if r.status_code == 200:
        json_gastos_parlamentares = json.loads(r.content)
        regex_equal_symbol = json_gastos_parlamentares["next"].rfind("=")
#         print(json_gastos_parlamentares["next"][regex_equal_symbol+1:])
#         print(json_gastos_parlamentares["results"][0]["numano"])
        df_gastos_parlamentares = pd.concat([df_gastos_parlamentares
                                             , pd.io.json.json_normalize(json_gastos_parlamentares["results"])]
                                            , axis=0
                                            , join="outer"
                                           )
    
print(df_gastos_parlamentares.shape)

df_gastos_parlamentares.sample(5).T

(100000, 29)


,154,569,737,457,51
codlegislatura,55,55,55,56,55
datemissao,None,None,None,None,None
idecadastro,74057,178830,178917,204436,160673
idedocumento,0,0,0,0,0
indtipodocumento,0,0,0,0,0
nucarteiraparlamentar,180,554,149,165,502
nudeputadoid,1531,2887,3006,3371,2281
nulegislatura,2015,2015,2015,2019,2015
numano,2015,2017,2017,2019,2017
numespecificacaosubcota,0,0,0,0,0


In [5]:
df_gastos_parlamentares.dropna(axis=1, inplace=True)
df_gastos_parlamentares.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 999
Data columns (total 24 columns):
codlegislatura             100000 non-null int64
idecadastro                100000 non-null int64
idedocumento               100000 non-null int64
indtipodocumento           100000 non-null int64
nucarteiraparlamentar      100000 non-null int64
nudeputadoid               100000 non-null int64
nulegislatura              100000 non-null int64
numano                     100000 non-null int64
numespecificacaosubcota    100000 non-null int64
numlote                    100000 non-null int64
nummes                     100000 non-null int64
numparcela                 100000 non-null int64
numressarcimento           100000 non-null int64
numsubcota                 100000 non-null int64
sgpartido                  100000 non-null object
sguf                       100000 non-null object
txnomeparlamentar          100000 non-null object
txtcnpjcpf                 100000 non-null object
txtdes

In [6]:
# Removendo os atributos com cardinalidade <= 1:

lista_atributos_para_remover = list(dp.cardinalidade(df_gastos_parlamentares)[dp.cardinalidade(df_gastos_parlamentares)["Cardinalidade"] <= 1]["Atributo"].values)

df_gastos_parlamentares.drop(lista_atributos_para_remover, axis=1, inplace=True)

In [7]:
df_gastos_parlamentares.vlrdocumento = pd.to_numeric(df_gastos_parlamentares.vlrdocumento)
df_gastos_parlamentares.vlrliquido = pd.to_numeric(df_gastos_parlamentares.vlrliquido)

df_gastos_parlamentares.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 999
Data columns (total 22 columns):
codlegislatura             100000 non-null int64
idecadastro                100000 non-null int64
idedocumento               100000 non-null int64
indtipodocumento           100000 non-null int64
nucarteiraparlamentar      100000 non-null int64
nudeputadoid               100000 non-null int64
nulegislatura              100000 non-null int64
numano                     100000 non-null int64
numespecificacaosubcota    100000 non-null int64
numlote                    100000 non-null int64
nummes                     100000 non-null int64
numparcela                 100000 non-null int64
numressarcimento           100000 non-null int64
numsubcota                 100000 non-null int64
sgpartido                  100000 non-null object
sguf                       100000 non-null object
txnomeparlamentar          100000 non-null object
txtcnpjcpf                 100000 non-null object
txtfor

**Análise preliminar:**

1. Os atributos / fatores / variáveis independentes que possuem cardinalidade igual a um não oferecem nenhum ganho de informação. Atributos 
**Ação:** Remover esses atributos.

2. Os atributos vlrdocumento e vlrliquido estão como objects, mas são floats.
**Ação:** Converter esses atributos para float.

In [8]:
dp.cardinalidade(df_gastos_parlamentares)

,Atributo,Cardinalidade,Valores
3,txtcnpjcpf,3,"[00000000000006, 00000000000001, 00000000000003]"
4,txtfornecedor,3,"[RAMAL, CELULAR FUNCIONAL, IMÓVEL FUNCIONAL]"
1,sguf,27,"[ES, SP, PI, SE, BA, PE, PR, RS, MG, SC, RJ, AC, PB, PA, AM, MA, CE, GO, MS, RO, RN, TO, MT, RR, AL, DF, AP]"
0,sgpartido,31,"[DEM, PP, PDT, PODE, PSB, PT, REPUBLICANOS, CIDADANIA, PSOL, MDB, PCdoB, PSD, PL, PROS, PSDB, PSC, PTB, PSL, SOLIDARIEDADE, NOVO, AVANTE, PATRIOTA, PV, REDE, PRB, PPS, PR, PATRI, PHS, PPL, PRTB]"
5,txtnumero,254,"[7009787, 6992781, 6984585, 6971935, 6971636, 6911066, 6900895, 6867342, 6860309, 6860198, 6858322, 6836040, 6764634, 6736164, 6727723, 6691809, 6700820, 6679940, 6679981, 6691831, 6679924, 6671376, 6679326, 6679406, 6679910, 6670073, 6679324, 6679906, 6669879, 6679318, 6679881, 6666957, 6679312, 6679870, 6665625, 6679296, 6679857, 6564475, 6679278, 6679656, 6501166, 6561355, 6679249, 6483697, 6482243, 6482268, 6484878, 6492209, 6497723, 6483711, 6482822, 6476873, 6459872, 6467744, 6475264, 6477598, 6477599, 6482863, 6476839, 6476876, 6476791, 6477811, 6468660, 6444265, 6444320, 6445187, 6455701, 6459000, 6476740, 6476806, 6460886, 6471575, 6468621, 6438055, 6438076, 6438104, 6438298, 6438649, 6476713, 6460610, 6471482, 6468580, 6433087, 6432835, 6432622, 6432765, 6438029, 6476711, 6459176, 6413347, 6413375, 6413380, 6413404, 6432079, 6476630, 6458936, 6342542, 6342545, 6348549, 6349350, ...]"
2,txnomeparlamentar,896,"[Norma Ayub, Fausto Pinato, Iracema Portella, Fábio Henrique, Bacelar, Felipe Carreras, Arlindo Chinaglia, Celso Russomanno, Gustavo Fruet, Rubens Bueno, Henrique Fontana, Pompeo de Mattos, Ivan Valente, Júlio Delgado, Rui Falcão, Angela Amin, Benedita da Silva, Leonardo Monteiro, Hermes Parcianello, Ricardo Barros, Renildo Calheiros, Sérgio Brito, Darcísio Perondi, Perpétua Almeida, Wellington Roberto, Bosco Costa, Alice Portugal, Daniel Almeida, Elcione Barbalho, José Priante, Átila Lins, Mário Heringer, Odair Cunha, Patrus Ananias, Reginaldo Lopes, Gastão Vieira, Carlos Sampaio, Gilberto Nascimento, Vicentinho, Leônidas Cristino, Roberto Pessoa, Júlio Cesar, Paes Landim, Nilson Pinto, Silas Câmara, João Campos, Rubens Otoni, Gonzaga Patriota, Vander Loubet, Giacobo, Maria do Rosário, Paulo Pimenta, Wolney Queiroz, Átila Lira, Damião Feliciano, André de Paula, Luciano Bivar, Claudio Cajado, José Rocha, Paulo Magalhães, Lincoln Portela, Aécio Neves, Eduardo Barbosa, Rodrigo Maia, Mauro Lopes, Luiza Erundina, Jandira Feghali, Maurício Dziedricki, Marcelo Freixo, Márcio Jerry, Chico D'Angelo, Dra. Soraya Manato, Pedro Augusto Bezerra, Santini, Leur Lomanto Júnior, Eduardo Bolsonaro, Fernando Monteiro, Stefano Aguiar, Hugo Leal, Cacá Leão, Lafayette de Andrada, Gleisi Hoffmann, Jaqueline Cassol, Benes Leocádio, Vinicius Carvalho, João Marcelo Souza, Gelson Azevedo, Darci de Matos, Adriano do Baldy, Pedro Paulo, Ronaldo Martins, Pedro Lucas Fernandes, Enio Verri, Lídice da Mata, André Figueiredo, Marcelo Moraes, Paulo Ramos, Edmilson Rodrigues, Cristiano Vale, Afonso Hamm, ...]"


In [9]:
df_gastos_parlamentares.groupby(["sguf", "sgpartido"], axis=0)["vlrliquido"].mean().sort_values(ascending=False)

sguf  sgpartido    
GO    DEM              280.395000
      PODE             251.564167
TO    PR               205.111613
GO    PTB              195.229781
DF    PP               185.859219
AL    MDB              171.860000
CE    PTB              165.495000
GO    MDB              150.207188
      PSC              142.540000
ES    DEM              138.992900
SP    CIDADANIA        137.364167
GO    PDT              130.609144
MT    PTB              126.816667
CE    PSDB             126.362745
      PSB              124.677626
SC    DEM              124.059149
PR    PROS             123.943286
MA    PROS             120.934545
GO    PT               119.444567
RO    PRB              117.396038
AM    PP               117.040000
CE    DEM              114.407169
MA    PV               113.851176
ES    PDT              112.609455
DF    PT               111.954100
AM    PRB              110.100105
MA    PRB              109.594740
GO    REPUBLICANOS     109.473333
MA    PSC              108.1